In [39]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

from catboost import Pool, CatBoostClassifier
from sklearn.metrics import roc_auc_score

Следующий блок добавлен на случай, если у вас проблемы с версиями polars, catboost, или в целом с .parquet файлами.

В случае конфликта версий catboost и polars при создании Pool, может потребоваться использовать fallback функцию to_pandas(). Более нигде pandas далее не используется.

In [40]:
import pandas as pd

try:
    import pyarrow
    print("PyArrow is installed. Version:", pyarrow.__version__)
except ModuleNotFoundError:
    print("PyArrow is NOT found in this environment. You installed it in a different one.")

PyArrow is installed. Version: 23.0.1


### Часть 1, смотрим данные

Сперва подгрузим данные и посмотрим с чем нам предстоит работать. 

Участникам предоставляются 2 пары файлов с признаками (train и test):

1. train_main_features.parquet + test_main_features.parquet (199 признаков + ключ)
2. train_extra_features.parquet + test_extra_features.parquet (2241 признак + ключ)


##### Часть 1.1 данные признаков

Попробуем сперва поработать с первым набором (*_main_features.parquet*).

In [41]:
train = pl.read_parquet('train_main_features.parquet')
test = pl.read_parquet('test_main_features.parquet')

print('Тренировочные данные:', train.shape)
print('Тестовые данные:', test.shape)

Тренировочные данные: (750000, 200)
Тестовые данные: (250000, 200)


In [43]:
train.head(n = 5)

customer_id,cat_feature_1,cat_feature_2,cat_feature_3,cat_feature_4,cat_feature_5,cat_feature_6,cat_feature_7,cat_feature_8,cat_feature_9,cat_feature_10,cat_feature_11,cat_feature_12,cat_feature_13,cat_feature_14,cat_feature_15,cat_feature_16,cat_feature_17,cat_feature_18,cat_feature_19,cat_feature_20,cat_feature_21,cat_feature_22,cat_feature_23,cat_feature_24,cat_feature_25,cat_feature_26,cat_feature_27,cat_feature_28,cat_feature_29,cat_feature_30,cat_feature_31,cat_feature_32,cat_feature_33,cat_feature_34,cat_feature_35,cat_feature_36,…,num_feature_96,num_feature_97,num_feature_98,num_feature_99,num_feature_100,num_feature_101,num_feature_102,num_feature_103,num_feature_104,num_feature_105,num_feature_106,num_feature_107,num_feature_108,num_feature_109,num_feature_110,num_feature_111,num_feature_112,num_feature_113,num_feature_114,num_feature_115,num_feature_116,num_feature_117,num_feature_118,num_feature_119,num_feature_120,num_feature_121,num_feature_122,num_feature_123,num_feature_124,num_feature_125,num_feature_126,num_feature_127,num_feature_128,num_feature_129,num_feature_130,num_feature_131,num_feature_132
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1610729,1.0,0.0,2.0,0.0,2.0,3.0,2.0,2.0,3.0,2.0,0.0,1.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,212.0,1.0,1.0,…,-0.284519,-0.264397,-0.004499,-0.150243,null,-0.002297,-0.043592,-0.061488,-0.264078,-0.009552,0.0,-0.26649,-0.060492,1.459966,null,-0.429813,null,-0.009654,3.569151,null,-0.256445,-0.008032,null,-0.24167,null,null,-0.001357,-0.031281,-0.046146,null,-0.418169,null,-0.057225,-0.107666,-0.45937,null,null
1239117,0.0,0.0,2.0,0.0,2.0,3.0,2.0,2.0,4.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,212.0,0.0,0.0,…,-0.284519,null,-0.004499,-0.150243,-0.503112,-0.002297,-0.043592,null,-0.145008,null,0.0,null,null,-0.279105,null,-0.429813,null,-0.009654,-0.293036,null,-0.57313,null,null,null,null,null,null,null,-0.046146,null,null,null,null,-0.170724,null,null,null
1737921,1.0,2.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,212.0,2.0,2.0,…,null,null,-0.004499,null,-0.810216,-0.002297,null,null,-0.264078,-0.009552,null,null,null,-0.279105,null,null,null,-0.009654,null,null,null,0.014575,null,null,null,null,null,null,null,null,null,null,null,-0.170724,null,null,null
1525853,1.0,0.0,2.0,1.0,2.0,3.0,2.0,2.0,4.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,2.0,0.0,0.0,212.0,0.0,0.0,…,null,null,-0.004499,null,null,null,null,-0.061488,null,-0.009552,null,null,-0.060492,null,null,null,null,null,null,null,null,null,null,null,-0.446954,null,-0.001357,-0.031281,null,-0.172704,-0.316505,null,null,null,-0.765018,null,null
1595130,1.0,0.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,212.0,0.0,0.0,…,null,-0.264397,-0.004499,null,null,-0.002297,null,-0.061488,3.546163,-0.009552,null,-0.26649,-0.060492,-0.279105,null,null,null,-0.009654,null,null,null,null,null,-0.505441,null,1.268075,-0.001357,-0.031281,null,null,-0.475778,0.048902,null,-0.170724,-0.765018,-0.397803,-0.373734


Легенда гласит, что в данных достаточно много пропусков. 

Так ли это?

In [44]:
missing_values = (
    train.null_count()
    .unpivot(variable_name="Column", value_name="Missing Values")
    .filter(pl.col("Missing Values") > 0)
    .sort("Missing Values", descending=True)
)

print('\nКоличество пропусков в признаках:\n', missing_values)


Количество пропусков в признаках:
 shape: (132, 2)
┌─────────────────┬────────────────┐
│ Column          ┆ Missing Values │
│ ---             ┆ ---            │
│ str             ┆ u32            │
╞═════════════════╪════════════════╡
│ num_feature_43  ┆ 149853         │
│ num_feature_54  ┆ 149810         │
│ num_feature_64  ┆ 149761         │
│ num_feature_34  ┆ 149661         │
│ num_feature_118 ┆ 149444         │
│ …               ┆ …              │
│ num_feature_27  ┆ 6193           │
│ num_feature_76  ┆ 3742           │
│ num_feature_62  ┆ 3741           │
│ num_feature_58  ┆ 3696           │
│ num_feature_98  ┆ 661            │
└─────────────────┴────────────────┘


Нам также известно, что часть признаков в данных — категориальная. 
По легенде их можно найти по названиям "cat_feature".

Эти категориальные признаки стоит отдельно выделить и сохранить для Catboost. 

In [45]:
cat_feature_names = [
    col_name for col_name in train.columns 
    if col_name.startswith("cat_feature")
]

In [46]:
train = train.with_columns(
    pl.col(cat_feature_names).cast(pl.Int32)
)


##### Часть 1.2 целевые переменные

В задаче предоставлена информация о 41 банковском продукте. 

Информацию о наличии каждого продукта требуется классифицровать по отдельности, в рамках multi-label постановки.

In [47]:
target = pl.read_parquet('train_target.parquet')

In [48]:
target.head(n = 5)

customer_id,target_1_1,target_1_2,target_1_3,target_1_4,target_1_5,target_2_1,target_2_2,target_2_3,target_2_4,target_2_5,target_2_6,target_2_7,target_2_8,target_3_1,target_3_2,target_3_3,target_3_4,target_3_5,target_4_1,target_5_1,target_5_2,target_6_1,target_6_2,target_6_3,target_6_4,target_6_5,target_7_1,target_7_2,target_7_3,target_8_1,target_8_2,target_8_3,target_9_1,target_9_2,target_9_3,target_9_4,target_9_5,target_9_6,target_9_7,target_9_8,target_10_1
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Давайте поверхностно посмотрим на то. с чем нам предстоит работать

In [49]:
target_columns = [col for col in target.columns if col.startswith("target")]

In [ ]:
target_col_means = (
    target.select(pl.col(target_columns).mean())
    .transpose(include_header = True, column_names = ["Mean"])
    .rename({"column": "Target"})
    #.sort("Sum", descending=True)
)

In [ ]:
plt.figure(figsize = (10, 6))

bars = plt.bar(target_col_means["Target"], target_col_means["Mean"])

plt.title("Средняя встречаемость целевой переменной", fontsize = 16, fontweight='bold')
plt.xlabel("Целевая переменная", fontsize = 12)
plt.ylabel("Средняя встречаемость", fontsize = 12)
plt.xticks(rotation = 45, ha = 'right', fontsize = 10)
plt.grid(axis = 'y', alpha = 0.3)

plt.tight_layout()
plt.show()

Выглядит интересно. И несбалансированно :)

Интересно, а что у них с совместной встречаемостью?

In [ ]:
target_row_sums = target.select(
    pl.sum_horizontal(target_columns).alias("row_sum")
)

target_row_sums.describe()

In [ ]:
target_corr_matrix = (
    target.select(target_columns)
    .corr()
    .to_numpy()
)

In [ ]:
plt.figure(figsize=(10, 9))
im = plt.imshow(target_corr_matrix, cmap = 'coolwarm', vmin = -1, vmax = 1, aspect = 'auto')

cbar = plt.colorbar(im, fraction = 0.046, pad = 0.04)
cbar.set_label('Коэффициент корреляции', fontsize = 12)

plt.title("Корреляционная матрица целевых переменных", fontsize = 16, fontweight = 'bold', pad = 20)
plt.xlabel("Целевая переменная", fontsize = 12)
plt.ylabel("Целевая переменная", fontsize = 12)

plt.xticks(np.arange(len(target_columns)), target_columns, rotation = 90, fontsize = 6)
plt.yticks(np.arange(len(target_columns)), target_columns, fontsize = 8)

plt.tight_layout()
plt.show()

Кажется, тут есть с чем можно работать. 

- На первый взгляд, есть выделяющаяся target_10_1 с отрицательной корреляцией к остальным целевым переменным.
- Внутри целевых переменных с 3 по 7 группу есть заметная внутри-групповая корреляция.
- Небольшая корреляция есть и между целевыми переменными из 1 и 2 групп, в частности между target_1_4 и target_2_2.

В дальнейшей работе это все можно было бы учесть и покопать глубже. Но сейчас нужно подготовить решение :)

### Часть 2, обучаем Catboost

К счастью для нас, Catboost умеет "из коробки" работать и с пропусками, и (важно для нас) с multi-class и multi-label классификацией.

Попробуем обучить модель "в лоб", оставив более интересные возможности и надстройки для участников соревнования :)


##### Часть 2.1 подготовка данных

- Опустим сам customer_id
- Не забудем про ранее сохраненные категориальные признаки
- Найдем в документации Catboost настройку "MultiLogLoss"
- Будем осторожны в первом подходе к снаряду, так как деревьев будет строиться много

In [ ]:
train_pool = Pool(data = train.drop("customer_id").to_pandas(), label = target.drop("customer_id").to_pandas(), cat_features = cat_feature_names)

In [ ]:
model = CatBoostClassifier(iterations = 1000, 
                           depth = 76, 
                           learning_rate = 0.05, 
                           loss_function = 'MultiLogloss', 
                           nan_mode = 'Min', 
                           random_seed = 1234,
                           verbose = 1,
                           thread_count=4)

Поехали :)

P.S. Это займет какое-то время, так что можно сходить попить чай

In [ ]:
model.fit(train_pool)

Настройку валидации и гиперпараметров оставим уже для участников соревнования 🤗

### Часть 3, подготовка сабмита

Посмотрим на пример рабочего бейзлайн решения. 

Именно в таком формате платформа ждет от нас решения:

In [ ]:
sample_submit = pl.read_parquet('sample_submit.parquet')

sample_submit.head(n = 5)

Посмотрели, осознали. Можем и сами все сделать :)

Сперва стоит сделать предсказания с помощью Catboost-а на тестовых данных. А для этого к ним стоит не забыть применить все те же манипуляции, что мы делали и с тренировочными данными (до обучения Catboost-а).

In [ ]:
test.head(n = 5)

In [ ]:
test = test.with_columns(
    pl.col(cat_feature_names).cast(pl.Int32)
)

In [ ]:
test_pool = Pool(data = test.drop("customer_id").to_pandas(), 
                 cat_features = cat_feature_names)

Делаем предсказания обученной ранее моделью. 

Предсказания будут в форме numpy.ndarray в 41 столбец.

In [ ]:
test_predict = model.predict(test_pool, prediction_type = "RawFormulaVal")

test_predict.shape

Пойдем по долгому пути и сами соберем свой submit файл.

In [ ]:
predict_schema = [col.replace("target_", "predict_") for col in target.columns if col.startswith("target_")]

catboost_predictions = pl.DataFrame(test_predict, schema = predict_schema)

catboost_predictions.head(n = 5)

In [ ]:
submit = test.select("customer_id")

submit = submit.hstack(catboost_predictions)

In [ ]:
submit.head(n = 5)

Выглядит похоже на правду :) 

Осталось только сохранить файл в .parquet и отправить на платформу, чтобы получить результат.

In [ ]:
submit.write_parquet("sample_submit.parquet")